In [7]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# https://ithelp.ithome.com.tw/articles/10218801
from google.colab import drive
# !cp /content/gdrive/***yourdata*** ./data
# !cp -R ./drive/'My Drive'/img2 ./img2  # <is this>
# !cp '/content/gdrive'./img2

In [20]:
def preprocess(filepath, imgf, labelf) :
  file = open(filepath, 'r')
  for line in file.readlines():
    img, label = line.split()
    imgf.append(img); labelf.append(label)
  file.close()

In [31]:
y = []
imgs = []
preprocess('/content/img2/tes.txt',imgs,y)

In [32]:
y

['0', '0', '0', '0', '0']

In [33]:
imgs

['img2/a1/n02111277_19.JPEG',
 'img2/a1/n02111277_25.JPEG',
 'img2/a1/n02111277_32.JPEG',
 'img2/a1/n02111277_39.JPEG',
 'img2/a1/n02111277_48.JPEG']

In [34]:
# https://stackoverflow.com/questions/28390614/opencv-hogdescripter-python
def cv2hog(imgl):
  histl = []
  winSize = (64,64)
  blockSize = (16,16)
  blockStride = (8,8)
  cellSize = (8,8)
  nbins = 9
  derivAperture = 1
  winSigma = 4.
  histogramNormType = 0
  L2HysThreshold = 2.0000000000000001e-01
  gammaCorrection = 0
  nlevels = 64
  for ig in imgl:
    image = cv2.imread(ig,0)
    # plt.imshow(image)
    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
    hist = hog.compute(image, winStride=(8,8), padding=(8,8), locations=((10,20),))
    histl.append(hist)
  return(histl)

In [35]:
print(imgs)
hists = cv2hog(imgs)
np.shape(hists[0])

['img2/a1/n02111277_19.JPEG', 'img2/a1/n02111277_25.JPEG', 'img2/a1/n02111277_32.JPEG', 'img2/a1/n02111277_39.JPEG', 'img2/a1/n02111277_48.JPEG']


(1764, 1)

In [36]:
np.save('tra_hog.npy', hists)

In [ ]:
# https://github.com/oreilly-japan/deep-learning-from-scratch/blob/master/common/layers.py
# https://github.com/oreilly-japan/deep-learning-from-scratch/blob/master/common/functions.py

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None # softmaxの出力
        self.t = None # 教師データ

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 教師データがone-hot-vectorの場合
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

def softmax(x):
    x = x - np.max(x, axis=-1, keepdims=True)   # overflow measures : looks like to reduce inf situations
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)

def softmax2(x):
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)

In [ ]:
x = np.random.normal(size=10)
print(x)
print(softmax(x))
print(softmax2(x))

[ 1.18306798  1.82109363  1.64577483 -0.74906252  0.97404695 -0.95554113
 -0.09205384 -0.12184826  1.42878427  1.0709846 ]
[0.12079858 0.22864035 0.19187256 0.01749638 0.09801333 0.01423232
 0.03375075 0.03276    0.15444544 0.10799028]
[0.12079858 0.22864035 0.19187256 0.01749638 0.09801333 0.01423232
 0.03375075 0.03276    0.15444544 0.10799028]


In [ ]:
x = np.array([1000000000000000000,-2,3,-4,5])
print(np.max(x, axis=-1, keepdims=True))
print(softmax(x))
print(softmax2(x))

[1000000000000000000]
[1. 0. 0. 0. 0.]
[nan  0.  0.  0.  0.]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
# https://github.com/PENGZhaoqing/Hog-feature/blob/master/hog.py
# https://github.com/karan6181/Softmax-Classifier/blob/master/softmaxClassifier.py
# https://github.com/Vercaca/Perceptron

# https://machinelearningmastery.com/softmax-activation-function-with-python/
# https://learnopencv.com/histogram-of-oriented-gradients/
# https://www.analyticsvidhya.com/blog/2019/09/feature-engineering-images-introduction-hog-feature-descriptor/
# https://openhome.cc/Gossip/CodeData/PythonTutorial/AssertDocTestPy3.html

# https://numpy.org/devdocs/user/quickstart.html

# read file
img = cv2.imread('/content/n02111277_19.JPEG', cv2.IMREAD_COLOR)
# plt.imshow(img, cmap=plt.cm.gray)
# print(img)
# np.shape(img)

plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

# hog = Hog_descriptor(img, cell_size=8, bin_size=8)
# vector, image = hog.extract()
# plt.imshow(image, cmap=plt.cm.gray)
# plt.show()

# Perceptron (Plot ?)

# Activation Function : Softmax
# https://zh.wikipedia.org/wiki/Softmax%E5%87%BD%E6%95%B0
z = np.array([1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 3.0])
print(np.exp(z)/sum(np.exp(z)))

In [ ]:
# https://stackoverflow.com/questions/28390614/opencv-hogdescripter-python
# https://docs.opencv.org/4.5.1/d5/d33/structcv_1_1HOGDescriptor.html#a5c8e8ce0578512fe80493ed3ed88ca83


<HOGDescriptor 0x7f968645ef10>


'\n#compute(img[, winStride[, padding[, locations]]]) -> descriptors\nwinStride = (8,8)\npadding = (8,8)\nlocations = ((10,20),)\nhist2 = hog2.compute(image,winStride,padding,locations)\nnp.shape(hist2)\n'

In [ ]:
from google.colab import drive
import os
# drive.mount('/content/gdrive')
!cp '/content/gdrive/My Drive/img2/'<img2>
!ls

/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `cp '/content/gdrive/My Drive/img2/'<img2>'
 gdrive		     sample_data    'train (1).txt'
 n02111277_19.JPEG  'test (1).txt'  'val (1).txt'


In [ ]:
from google.colab import files
!zip -r /content/imagesf.zip /content/imagesf
files.download("/content/imagesf.zip")

updating: content/imagesf/ (stored 0%)
updating: content/imagesf/a1/ (stored 0%)
updating: content/imagesf/a1/n02111277_25.JPEG (deflated 1%)
updating: content/imagesf/a1/n02111277_39.JPEG (deflated 1%)
updating: content/imagesf/a1/n02111277_19.JPEG (deflated 1%)
updating: content/imagesf/a1/n02111277_48.JPEG (deflated 1%)
updating: content/imagesf/a1/n02111277_32.JPEG (deflated 2%)
updating: content/imagesf/a2/ (stored 0%)
updating: content/imagesf/a2/n02111277_115.JPEG (deflated 1%)
updating: content/imagesf/a2/n02111277_101.JPEG (deflated 1%)
updating: content/imagesf/a2/n02111277_94.JPEG (deflated 1%)
updating: content/imagesf/a2/n02111277_104.JPEG (deflated 2%)
updating: content/imagesf/a2/n02111277_110.JPEG (deflated 2%)
updating: content/imagesf/tes.txt (deflated 68%)
updating: content/imagesf/tra.txt (deflated 74%)
  adding: content/imagesf/.ipynb_checkpoints/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# try perception

class Perceptron:
  def __init__(self, n_inputs, activ_func='Maxout', save_fig=True, batch_size=16, learning_rate=0.01, n_iters=1000):
      self.__weights = np.array([0.0] * (n_inputs + 1)) # 1 for bias

      # self.activation_func = self.activation
      self.n_iters = n_iters  # total epochs
      self.lr = learning_rate
      self.epoch = 1
  @property
  def train(self, X, y):


'''
def train(self, datasets):
        n_iteration = 0
        n_weights = len(self.__weights)
        if len(datasets.columns) != n_weights:
            raise Exception("Wrong inputs of training!")

        x_title, y_title = list(datasets.columns)[:2]
        # check_error_results
        result = self.check_error(datasets, n_iteration)

        while result:
            n_iteration += 1
            X, target = result
            self.update_weights(X, target)

            plot_data_and_line(datasets, W=self.weights, iter_time=n_iteration, x_title=x_title, y_title=y_title, save_fig=self.__save_fig)
            result = self.check_error(datasets, n_iteration)

def check_error(self, datasets, n_iteration):
        n_weights = len(self.__weights)
        error = 0
        result = False
        for i in range(datasets.shape[0]):
            row = datasets.iloc[i].tolist()
            X, target = np.array([1] + row[:-1]), row[-1]  # initial x[0] as 1, for bias, so that w0*x0 = w0 (aka. b)

            y = self.activation(np.dot(self.__weights, X)) # W*X = w0*x0 + w1*x1 + ... + w_n*x_n

            if target != y:
                error += 1
                result = X, target
        print('Iteration #{}: error = {}'.format(n_iteration, error))
        return result
'''
  def get_loss(self, y_hat, y):
    # multi-class cross-entropy
    loss += np.dot(y ,np.lop(y_hat))  #??
    
    print('Epoch #{}: loss = {}'.format(self.epoch, loss))
    return loss
  
  def top_accuracy(self, y_hat, y):
    batch_n = y.shape[0] #??
    top1 = np.sum(np.equal(np.array([1,2,3,4,5]),np.array([1,3,5,7,9]))) ##
    top5 = np.sum(np.isin(element,np.sort(np.argpartition(arr, len(arr) - k)[-k:]))) ##

    return top1, top5

  def train(self, X, y):
    n_weights = len(self.__weights)

    if len(X.columns) != n_weights:
      raise Exception("Wrong inputs of training!")

    n_weights = len(self.__weights)
    loss = 0
    for i in range(X.shape[0]):
      data = np.array([1] + X.iloc[i].tolist())
      y_hat = activation(np.dot(self.__weigths, X))
    loss = get_loss(y_hat, y)
    top1_acc, top5_acc = top_accuracy(y_hat, y)

  def weights(self):
      return self._weights[:]
  
  def update_weights(self, X:list, y:int):
    self.lr
  
  def activation(self, wx): # maxout
      y_hat = lambda wx: np.exp(wx)/sum(np.exp(wx))
      return y_hat = lambda y: np.exp(y)/sum(np.exp(y))

# .npy


In [ ]:
from sklearn import datasets
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append('../../ml')

from utils import plot_data_and_line
from activation.activation import ActivationFunction
from preprocess.iris_preprocess import iris_data_preprocess

class Perceptron:
    def __init__(self, n_inputs, activ_func='Sign', save_fig=False):
        self.__weights = np.array([0.0] * (n_inputs + 1)) # 1 more for bias
        self.__save_fig = save_fig
        self.__activation = ActivationFunction(activ_func)

    @property
    def weights(self):
        return self.__weights[:]

    def update_weights(self, X:list, y:int):
        print('>> Updating')

        self.__weights += X*y   # w += delta_w --> y * X
        print('after update: {}'.format(self.__weights))

    def activation(self, y): # sign
        return self.__activation.func(y)
        # if y > 0:
        #     return 1
        # else:
        #     return -1

    def check_error(self, datasets, n_iteration):
        n_weights = len(self.__weights)
        error = 0
        result = False
        for i in range(datasets.shape[0]):
            row = datasets.iloc[i].tolist()
            X, target = np.array([1] + row[:-1]), row[-1]  # initial x[0] as 1, for bias, so that w0*x0 = w0 (aka. b)

            y = self.activation(np.dot(self.__weights, X)) # W*X = w0*x0 + w1*x1 + ... + w_n*x_n

            if target != y:
                error += 1
                result = X, target
        print('Iteration #{}: error = {}'.format(n_iteration, error))
        return result

    def train(self, datasets):
        n_iteration = 0
        n_weights = len(self.__weights)
        if len(datasets.columns) != n_weights:
            raise Exception("Wrong inputs of training!")

        x_title, y_title = list(datasets.columns)[:2]
        # check_error_results
        result = self.check_error(datasets, n_iteration)

        while result:
            n_iteration += 1
            X, target = result
            self.update_weights(X, target)

            plot_data_and_line(datasets, W=self.weights, iter_time=n_iteration, x_title=x_title, y_title=y_title, save_fig=self.__save_fig)
            result = self.check_error(datasets, n_iteration)


def main():
    # read data and preprocessing
    iris_data = iris_data_preprocess()

    # build model
    myPerceptron = Perceptron(n_inputs=len(iris_data.columns)-1, save_fig=False)
    myPerceptron.train(iris_data)

    # plot the result
    x_title, y_title = list(iris_data.columns)[:2]
    plot_data_and_line(iris_data, myPerceptron.weights, x_title=x_title, y_title=y_title, save_fig=False)

if __name__ == '__main__':
    main()